In [27]:
%matplotlib inline

In [121]:
import unittest
from unis.models import *
from unis.runtime import Runtime
import networkx as nx
import itertools

In [2]:
def test(case=None):
    import sys
    if case is None:
        module = sys.modules[__name__]
        suite = unittest.TestLoader().loadTestsFromModule(module)
    else:
        suite = unittest.TestLoader().loadTestsFromTestCase(case)
    unittest.TextTestRunner().run(suite)

In [3]:
class FlangeTree(object):
    def __call__(self): 
        raise RuntimeError("Must override the call method")

class ActionFailureError(RuntimeError): 
    "Inidicates rule action was taken BUT the test still fails"
    pass

        
def update(graph, values): pass

In [4]:

class rule(FlangeTree): 
    """
    A test/action pair. Executes the action if the rule fails.
    Returns the result of action (if taken) or the original arguments (if no action taken).
    
    The result of the action must be a list of things of the same type as the args.
    This makes the test/action/retest work properly.
    """

    def __init__(self, test, action):
        self.test = test
        self.action = action
        
    def __call__(self, *args):
        rslt=args
        if not self.test(*args): 
            rslt = self.action(*args)
            if not self.test(*rslt):
                raise ActionFailureError()
            return rslt   
        return rslt
            
class Test_rule(unittest.TestCase):
    def test_action_pass(self):
        c = rule(lambda x: 4<x[0], lambda x: [[6]])
        self.assertEqual(c([3]), [[6]])

    def test_simple_pass(self):
        c = rule(lambda x: 4>3, lambda x: [[3]])
        self.assertEqual(c(5), (5,))
        
    def test_double_fail(self):
        c = rule(lambda *x: 4<3, lambda *x: [4])
        self.assertRaises(ActionFailureError, c)
        

In [5]:
class switch(object):
    "A rule that always tests to True."
    def default(action): return rule(lambda *x: True, action)

    "Tests rules in order. Does the action associated with the first item test that passes."
    def __init__(self, *rules):
        self.rules = rules

    def __call__(self):
        for rule in self.rules:
            if rule.test():
                return rule.action()
        raise RuntimeError("No action take in switch")

class Test_switch(unittest.TestCase):
    FALSE = lambda *x: False
    TRUE = lambda *x: True
    
    def test_default(self):
        s = switch(rule(self.FALSE, lambda *x: None), 
                   rule(self.FALSE, lambda *x: None), 
                   rule(self.FALSE, lambda *x: None), 
                   switch.default(lambda *x: 10))
        self.assertEqual(s(), 10)
        
    def test_intermediate(self):
        s = switch(rule(self.FALSE, lambda *x: None), 
                   rule(self.TRUE, lambda *x: 6), 
                   rule(self.FALSE, lambda *x: None), 
                   switch.default(lambda *x: 10))
        self.assertEqual(s(), 6)
        
    def test_default(self):
        c = switch.default(lambda *x: 3)
        self.assertEqual(c.test(), True)
    


In [13]:
class unis(object):
    "Retrieves a graph from a UNIS server."
    default_unis = "http://192.168.100.200:8888"
    _runtime_cache = {}
    
    def __init__(self, ref="*", source=default_unis):
        self.source = source
        self.ref = ref

    def __call__(self):
        rt = self.cached_connection(self.source)
        
        #TODO: This is a bare-bones buildling of the graph model
        if self.ref == "*": topology = rt
        else: topology= list(rt.topologies.where({"id": self.ref}))[0]

        g = nx.Graph() #TODO: Digraph?
        for port in topology.ports:
            g.add_node(port)
        for link in topology.links:
            g.add_edge(link.endpoints[0], link.endpoints[1], object=link)
        
        return g
    
    @classmethod
    def cached_connection(cls, source):
        rt = cls._runtime_cache.get(source, Runtime(source))
        cls._runtime_cache[source] = rt
        return rt

class Test_unis(unittest.TestCase):
    explicit_host = "http://192.168.100.200:8888"
    
    def setUpClass():
        unis._runtime_cache = {}
        print("Cleared unis Runtime cache")

    def test_implict_host(self):
        b = unis()
        g = b()
        self.assertEqual(len(g.nodes()), 2)
        self.assertEqual(len(g.edges()), 1)

    def test_all_implicit(self):
        b = unis(source=self.explicit_host)
        g = b()
        self.assertEqual(len(g.nodes()), 2)
        self.assertEqual(len(g.edges()), 1)
        
    def test_all_explicit(self):
        b = unis("*", self.explicit_host)
        g = b()
        self.assertEqual(len(g.nodes()), 2)
        self.assertEqual(len(g.edges()), 1)
        
    def test_named(self):
        b = unis("test", self.explicit_host)
        g = b()
        self.assertEqual(len(g.nodes()), 2)
        self.assertEqual(len(g.edges()), 1)

In [7]:
class GroupCondition(FlangeTree):
    def __init__(self, predicate, selector, graph): 
        self.predicate = predicate
        self.selector = selector
        self.graph = graph
    
    def __call__(self, *args):
        g = self.graph()
        raw_items = self.selector(g)
        passing_items = [x for x in raw_items if self.predicate(x, g)]
        return self._test(passing_items, raw_items)
    
    def _test(self, passing_items, raw_items): 
        raise Error("Not implemented")
    
class exists(GroupCondition):
    def _test(self,  passing_items, raw_items): 
        return len(passing_items) > 0
    
class exactlyOne(GroupCondition):
    def _test(self, passing_items, raw_items): 
        return len(passing_items) == 1
    
class all(GroupCondition):
    def _test(self, passing_items, raw_items): 
        return len(passing_items) == len(raw_items)

class most(GroupCondition):
    def _test(self, passing_items, raw_items): 
        return len(passing_items) >= len(raw_items)//2
    
class TestGroupConditions(unittest.TestCase):
    def test_exists_pass(self):
        t = exists(lambda n, g: g.degree(n) > 0, 
                   lambda g: g.nodes(), 
                   unis())
        self.assertTrue(t())
        
    def test_exists_fail(self):
        t = exists(lambda n, g: g.degree(n) == 0, 
                   lambda g: g.nodes(), 
                   unis())
        self.assertFalse(t())

    def test_exactlyone_pass(self):
        t = exists(lambda n, g: n.id == "port1", 
                   lambda g: g.nodes(), 
                   unis())
        self.assertTrue(t())
        
    def test_exactlyOne_fail(self):
        t = exactlyOne(lambda n, g: g.degree(n) == 0, 
                   lambda g: g.nodes(), 
                   unis())
        self.assertFalse(t())
        
        t = exactlyOne(lambda n, g: g.degree(n) > 0, 
                   lambda g: g.nodes(), 
                   unis())
        self.assertFalse(t())
            
    def test_all_pass(self):
        t = all(lambda n, g: g.degree(n) > 0, 
                   lambda g: g.nodes(), 
                   unis())
        self.assertTrue(t())

    def test_all_fail(self):
        t = all(lambda n, g: n.id == "port1", 
                   lambda g: g.nodes(), 
                   unis())
        self.assertFalse(t())

    
    def test_most_pass(self): 
        t = most(lambda n, g: n.id == "port1",
                 lambda g: g.nodes(), 
                 unis())
        self.assertTrue(t())
        
    def test_most_fail(self):
        t = most(lambda n, g: n.id == "port3",
                 lambda g: g.nodes(), 
                 unis())
        self.assertFalse(t())



In [8]:
class place(FlangeTree):
    def __init__(self, mod, at, graph):
        self.mod = mod
        self.at = at
        self.graph = graph
    
    def __call__(self, *args):
        g = self.graph()
        position = self.at(g)
        g2 = self.mod(position, g)
        return g2

class Test_place(unittest.TestCase):
    def test_placement(self):
        p = place(lambda positions, g: {n.id: "modified" for n in positions},
                  lambda g: g.nodes(),
                  unis())
        self.assertEqual(p(), {"port1": "modified", "port2": "modified"})


In [117]:
def inside(selector, graph):
    """Given a graph and a selector, returns a list of links that make up the min cut"""
    nodes = list(filter(selector, graph.nodes())) ## TODO -- More complex query here
    synth = graph.subgraph(nodes) ## Get the subgraph indicated by the selector
    nx.set_edge_attributes(synth, 'capacity', 1)

    #outbound = graph.out_edges_iter(nodes)  -- Switch unis to directed graph, then use
    #inbound = graph.in_edges_iter(nodes)
    outbound = graph.edges_iter(nodes)
    inbound = graph.edges_iter(nodes)


    src = "##SOURCE##"
    synth.add_node(src)
    for edge in inbound:
        synth.add_edge(src, edge[1], capacity=10)

    sink = "##SINK##"
    synth.add_node(sink)
    for edge in outbound:
        synth.add_edge(edge[0], sink, capacity=10)

    cut_value, partition = nx.minimum_cut(synth, s=src, t=sink, capacity='capacity')
    reachable, non_reachable = partition
    
    cutset = set()
    for u, nbrs in ((n, synth[n]) for n in reachable):
        cutset.update((u, v) for v in nbrs if v in non_reachable)
    cut_value == sum(synth.edge[u][v]['capacity'] for (u, v) in cutset)
    
    return cutset

class Test_inside(unittest.TestCase):
    def test(self):
        i = inside(lambda x: x, unis()())
        r = {(e[0].id, e[1].id) for e in cut}
        self.assertEqual(r, {("port1", "port2")})

In [136]:
def around(selector, graph): 
    """Given a graph and a selector, returns a list of links that go in or out of the selected nodes"""
    nodes = list(filter(selector, graph.nodes())) ## TODO -- More complex query here
    print(nodes)
    outbound = graph.edges_iter(nodes)
    inbound = graph.edges_iter(nodes)
    return set(itertools.chain(outbound, inbound))

class Test_around(unittest.TestCase):
    def test(self):
        i = around(lambda x: x.id == "port1", unis()())
        r = {(e[0].id, e[1].id) for e in cut}
        self.assertEqual(r, {("port1", "port2")})

In [ ]:
def near(fn, selector, graph): pass
def between(selector, graph): pass

In [116]:
test()

...................

Cleared unis Runtime cache



----------------------------------------------------------------------
Ran 19 tests in 0.217s

OK


In [19]:
def populate(url):
  with Runtime(url) as rt:
    node1 = Node({"id": "node1"})
    node2 = Node({"id": "node2"})
    port1 = Port({"id": "port1"})
    port2 = Port({"id": "port2"})
    node1.ports.append(port1)
    node2.ports.append(port2)
    link1 = Link({"id": "link1-2", "directed": False, "endpoints": [port1, port2]})
    topology = Topology({"id": "test", "ports" : [port1, port2], "nodes": [node1, node2], "links" : [link1]})
    rt.insert(port1, commit=True)
    rt.insert(port2, commit=True)
    rt.insert(node1, commit=True)
    rt.insert(node2, commit=True)
    rt.insert(link1, commit=True)
    rt.insert(topology, commit=True)

In [20]:
populate("http://192.168.100.200:8888")